# Build a DF relating item category name and shop name to shop ids and item ids

In [1]:
TARGET_ENCODING_NPERM = 8
KFOLD_K = 5
KFOLD_ALPHA=0

### Load packages

In [2]:
import pandas as pd
import pickle
import numpy as np
import re
import gc
from sklearn import preprocessing
from sklearn import model_selection

### Read in Data

In [3]:
df_item_cat = pd.read_csv('../gen_data/train2.csv')

y_train_list = pickle.load(open('../gen_data/y_train--features1.ipynb--.pickle','rb'))
x_train_list = pickle.load(open('../gen_data/x_train--features3.ipynb--.pickle','rb'))
x_test_list = pickle.load(open('../gen_data/x_test--features3.ipynb--.pickle','rb'))

sales = pd.read_csv('../original_data/sales_train.csv.gz')

In [4]:
df_city = df_item_cat.copy()

# Generate new item variable

In [5]:
#start with sales data, figure out when each item was sold, 
    #merge this with train/test then transform it to 1/0
#fillna with new item?
def new_item_var(df):
    sales2 = sales.copy()
    first_sale_date = sales2.groupby('item_id').date_block_num.min()
    sales2['first_sale_date_block'] = sales2.item_id.map(first_sale_date)
    sales2 = sales2[['item_id','first_sale_date_block']]
    sales2 = sales2.drop_duplicates()
    df = df.merge(sales2,on='item_id',how='left')
    
    #handle nans for test data (which sales2 wont pick up)
    df.first_sale_date_block.fillna(999,inplace=True)    
    df['new_item'] = (df.first_sale_date_block >= df.date_block_num)*1

    return df

In [6]:
x_train_list = list(map(new_item_var,x_train_list))
x_test_list = list(map(new_item_var,x_test_list))
del sales
gc.collect()

216

### Parse City data

In [8]:
#cities_encoder = preprocessing.LabelEncoder()
#cities_series = pd.Series([re.search('(.*?) ',n).group() for n in df.shop_name],index=df.index)
#cities_encoder.fit(cities_series)
#df['city'] = cities_encoder.transform(cities_series)
#del cities_series

cities_series = pd.Series([re.search('(.*?) ',n).group() for n in df_city.shop_name],index=df_city.index)
df_city['city'] = cities_series

In [9]:
np.mean(cities_series.isna())

0.0

### Parse item category data

In [10]:
#item_type_encoder = preprocessing.LabelEncoder()
#item_type_series = df.item_category_name.map(lambda x: re.split('-',x)[0])
#item_type_encoder.fit(item_type_series)
#df['item_type'] = item_type_encoder.transform(item_type_series)
#del item_type_series

item_type_series = df_item_cat.item_category_name.map(lambda x: re.split('-',x)[0])
df_item_cat['item_type'] = item_type_series

### parse more item_category data

In [11]:
#item_info_encoder = preprocessing.LabelEncoder()
#item_info_series = df.item_category_name.map(lambda x: '-'.join(re.split('-',x)[1:]) if len(re.split('-',x))>1 else 'NAN')
#item_info_encoder.fit(item_info_series)
#df['item_info'] = item_info_encoder.transform(item_info_series)
#del item_info_series

item_info_series = df_item_cat.item_category_name.map(lambda x: '-'.join(re.split('-',x)[1:]) if len(re.split('-',x))>1 else 'NAN')
df_item_cat['item_info'] = item_info_series

### Select relevant columns in df

In [12]:
df_city = df_city[['shop_id','city']].drop_duplicates()
df_item_cat = df_item_cat[['item_category_id','item_type','item_info']].drop_duplicates()

### function to downcast data types to 32 bits

In [13]:
def downcast(df):
    float_cols = [col for col in df if df[col].dtype=='float64']
    int_cols = [col for col in df if df[col].dtype=='int64']

    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int32)
    
    return(df)

### cleanup

In [14]:
df_city = downcast(df_city)
df_item_cat = downcast(df_item_cat)
gc.collect()

7

### Merge data above into training and test sets

In [15]:
x_train_list_saved = x_train_list

In [16]:
d = x_train_list_saved[0].sample(frac=0.4)
print(d.shape)
#print(df.shape)
#d.merge(df,on=['item_category_id'],how='left')

(2474769, 72)


NameError: name 'df' is not defined

In [20]:
#df_item_cat.shape

(84, 3)

In [18]:
x_train_list = list(map(lambda x: x.merge(df_item_cat,on=['item_category_id'],how='left'),x_train_list))
x_test_list = list(map(lambda x: x.merge(df_item_cat,on=['item_category_id'],how='left'),x_test_list))

In [21]:
x_train_list = list(map(lambda x: x.merge(df_city,on=['shop_id'],how='left'),x_train_list))
x_test_list = list(map(lambda x: x.merge(df_city,on=['shop_id'],how='left'),x_test_list))

# check 3

In [30]:
tocheck = x_train_list[1]
print(np.mean(tocheck.city.isna()))
print(np.mean(tocheck.item_type.isna()))
print(np.mean(tocheck.item_info.isna()))
print(np.mean(tocheck.item_info=='NAN'))

0.0
0.0
0.0
0.0029689838000813685


# check 2

In [22]:
np.mean(x_train_list[1].city.isna())

0.0

In [23]:
#np.mean(df.city.isna())

NameError: name 'df' is not defined

In [24]:
new = x_train_list_saved[1][['date_block_num','shop_id','item_id','item_category_id']].merge(df,how='left')
#new.head()
print(np.mean(new.city.isna()))
new.head()

NameError: name 'df' is not defined

In [ ]:
newna = new[new.city.isna()]
newna.head()

In [25]:
new[(new.shop_id==2)]

NameError: name 'new' is not defined

In [ ]:
#df doesnt contain every shop item combo so when i merge it is misssing a lot,
#need to merge separatl on shop and item

In [ ]:
x_train_list_saved[1].head()

In [ ]:
df.head()

# check

In [ ]:
x_train_list[1].city.unique()
#x_test_list[1].city.unique()
np.sum(cities_series=='!Якутск ')
df.city.unique()

In [ ]:
df[df.city=='!Якутск '].head()

In [26]:
tc = x_train_list[1]
tc[(tc.shop_id==0)].head()

,date_block_num,shop_id,item_id,item_category_id,sum_item_sales_back_1,sum_shop_sales_back_1,item_cnt_month_back_1,sum_item_cat_sales_back_1,sum_item_cat_shop_sales_back_1,sum_item_sales_back_2,...,numdays,mean_prevmonth_item_price,sd_prevmonth_item_price,mean_prevmonth_shop_item_price,sd_above_market,first_sale_date_block,new_item,item_type,item_info,city


In [27]:
tc[['item_category_id','city']]
np.mean(df.city.isna())

NameError: name 'df' is not defined

In [28]:
np.mean(tc.city.isna())


0.0

In [ ]:
#should be 0
print(np.sum(tc.city.isna()))
#should be 0
print(np.sum(tc.item_type.isna()))
#i would imagine should be small, sometimes info isnt parsed
print(np.mean(tc.item_info=='NAN'))

In [ ]:
[print(x) for x in df.city.unique()]
print('\n\n')
[print(x) for x in df.item_type.unique()]
print('\n\n')
[print(x) for x in df.item_info.unique()]

### it is possible (should check) that all of the !Якутск transactions were early and thus not picked up, should still fix because datebackgen will change

In [ ]:
np.sum((df.city=='!Якутск '))

In [ ]:
tc[(tc.shop_id==50)].head()

In [ ]:
df[(df.item_id==30) & (df.shop_id==2)]

In [ ]:
tc.item_category_id.unique().min()

In [ ]:
tc[tc.city.isna()][['item_id','item_category_id','shop_id']]

# Target Encoding

Target encoding is based on historical information, available for both train and test datasets, as such, we needn't worry about regularisation specifically for target encodings.

In [ ]:
def kfold_target_encode(x_train,y_train,x_test,cols,k=5,alpha=0):
    x_train['target'] = y_train
    globmean = x_train.target.mean()
#for test data
    for col in cols:
        agged_sum = x_train.groupby(col).target.sum()
        x_test[col+'_encoded'] = x_test[col].map(agged_sum)
        x_test[col+'_encoded'].fillna(globmean,inplace=True)
            
#for train data, need to regularise
    #ensure index is reset to avoid breaking later
    

    x_train.reset_index(drop=True,inplace=True)
    x_train[col+'_encoded']=np.nan
    for col in cols:
        print('Encoding '+col)
        folds = model_selection.KFold(n_splits=k,shuffle=True,random_state=0)
        for compute_ind , map_ind in folds.split(x_train):
            mean = x_train.iloc[compute_ind].groupby(col).target.mean()
            count = x_train.iloc[compute_ind].groupby(col).target.count()
            encoding = (mean*count + globmean*alpha)/(alpha+count)
            x_train.loc[map_ind,col+'_encoded'] = x_train.loc[map_ind,col].map(encoding) 
            x_train[col+'_encoded'].fillna(globmean,inplace=True)
    return x_train.drop('target',axis=1) , x_test

In [ ]:
def perm_target_encode(x_train,y_train,x_test,cols,nperms):
    x_train['target'] = y_train
    print('top: '+str(x_train.shape))
#for test data
    for col in cols:
        agged_sum = x_train.groupby(col).target.sum()
        x_test[col+'_encoded'] = x_test[col].map(agged_sum)
        x_test[col+'_encoded'].fillna(0,inplace=True)
            
#for train data, need to regularise
    for col in cols:
        encoding_df = pd.DataFrame()
        for perm_num in range(0,nperms):
            print('\tComputing encoding on permutation '+str(perm_num)+' of '+str(nperms))
            #set random state to perm num for reproducibility
            #make this faster by just changing index?
            perm_x_train = x_train.sample(frac=1,random_state=perm_num)
            cumsum   = perm_x_train.groupby(col).target.cumsum() - perm_x_train.target
            cumcount = perm_x_train.groupby(col).target.cumcount()
            encoding = cumsum/cumcount
            
            encoding_df['perm_'+str(perm_num)+'_encoding'] = encoding
            
        encoding_df['colmeans'] = encoding_df.mean(axis=1)    
        x_train[col+'_encoded'] = encoding_df.colmeans
        x_train[col+'_encoded'].fillna(0,inplace=True)
        del encoding_df
        gc.collect()
    return x_train.drop('target',axis=1) , x_test

In [ ]:
to_target_encode = ['shop_id','item_id','item_category_id','month','city','item_type','item_info']
to_target_encode=['shop_id']
jumbled_list = [kfold_target_encode(x_train,y_train,x_test,to_target_encode,KFOLD_K,alpha=KFOLD_ALPHA) for x_train , y_train, x_test in zip(x_train_list,y_train_list,x_test_list)]
x_train_list , x_test_list = list(map(list,zip(*jumbled_list)))

In [ ]:
d = x_train_list[1]
d['target'] = y_train_list[1]

In [ ]:
d[d.shop_id==50].shop_id_encoded.unique()

In [ ]:
x_train_list[0]['shop_id_encoded']

In [ ]:
d.groupby('shop_id').target.mean()

### Save

In [ ]:
pickle.dump(x_train_list,open('../gen_data/x_train--features4.ipynb--.pickle','wb'))
pickle.dump(x_test_list,open('../gen_data/x_test--features4.ipynb--.pickle','wb'))

In [ ]:
x_train_list[0].head()

In [ ]:
#to_target_encode = ['shop_id','item_id','item_category_id','month','city','item_type','item_info']
#jumbled_list = [perm_target_encode(x_train,y_train,x_test,to_target_encode,TARGET_ENCODING_NPERM) for x_train , y_train, x_test in zip(x_train_list,y_train_list,x_test_list)]
#x_train_list , x_test_list = list(map(list,zip(*jumbled_list)))

In [ ]:
# need to add bool: item never sold before
                    #shop never sold before
# and int: num item sold
# num shop sold
# FOR EACH MONTH

#this is target encoding filling nans with 0 except missing early data

#doing this early in pipeline would be easy

#going back for all time might be a bad idea, the nature of the data would change thru time, (summingup)
#instaed just use dateback range

In [ ]:
#create variables to quantify how much we know about a data point
#For each shop_id and item_id create sum of recent history(total sales for dateback_range)
#create bool if 0 indicating new shop or item
def generate_recent_history_summary(df):
    df_items = df.filter(regex='sum_item_sales_back_\d+$')
    df['historical_item_sales'] = df_items.sum(axis=1)
    df['new_item'] = (df.historical_item_sales<0.5)*1
    
    df_shops = df.filter(regex='sum_shop_sales_back_\d+$')
    df['historical_shop_sales'] = df_shops.sum(axis=1)
    df['new_shop'] = (df.historical_shop_sales<0.5)*1
    
    df_item_cats = df.filter(regex='sum_item_cat_sales_back_\d+$')
    df['historical_item_cat_sales'] = df_item_cats.sum(axis=1)
    df['new_item_cat'] = (df.historical_item_cat_sales<0.5)*1
    return df
    

In [ ]:
x_train_list = list(map(generate_recent_history_summary,x_train_list))
x_test_list = list(map(generate_recent_history_summary,x_test_list))

In [ ]:
x_train_list[0][['item_id','historical_item_sales']]

In [ ]:
df = x_train_list[0][['date_block_num','item_id','item_id_encoded','historical_item_sales']].copy()
df.columns = ['date_block_num','item_id','ORIGINAL_item_id_encoded','historical_item_sales']
df.head()

In [ ]:
#work on this
#this isnt working properly, drop it, I have almost the same information from elsewhere
def old_target_encode(df,target,cols,fillna=0):
    maxmonth = df.date_block_num.max()
    print(maxmonth)
    df['target'] = target
    for col in cols:
        print('Encoding variable: '+col)
        #dont use current data, that way we can do this for both train and test sets
        agged_targ = df[df.date_block_num<maxmonth].groupby(col).target.sum()
        agged_targ = df[(df.date_block_num<maxmonth) & (df.date_block_num>=(maxmonth-12))].groupby(col).target.sum()
        #agged_targ = df[df.date_block_num>20].groupby(col).target.sum()
        
        
        
        df[col+'_encoded'] = df[col].map(agged_targ)
        if fillna=='mean':
            df[col].fillna(df(col).mean(),inplace=True)
        elif fillna=='median':
            df[col].fillna(df(col).median(),inplace=True)
        else:
            df[col].fillna(0,inplace=True)
    return df.drop('target',axis=1)

### Im counting back the months from max month, but historical is rolling for each date, historical is bad becuase it double counts, or does it?

In [ ]:
#targs = y_train_list[0]
#target_encode(df,targs,['item_id'])
#g = df[(df.date_block_num<32) & (df.date_block_num>=20)].groupby('item_id').target.sum()
#df['item_id_encoded_manual'] = df.item_id.map(g)
#df.head()

In [ ]:
21 ... 31

In [ ]:
df = x_train_list[1]
targs = y_train_list[1]
df['target'] = targs
df_small = df[['item_id','item_id_encoded','target','historical_item_sales']]

In [ ]:
g = df.groupby('item_id').target.sum()
g.head(40)

In [ ]:
df_small['item_id_encoded_manual'] = df.item_id.map(g)

In [ ]:
df_small.head()

In [ ]:
df_small.head(15)

In [ ]:
df[(df.item_id==32)].target.sum()

In [ ]:
#df[['item_id','item_idencoded']].sort_values('item_id').head(1000)

In [ ]:
df100 = df.sample(100)
df100